In [ ]:
!pip install numpy opencv-python pillow piexif pytz astropy pandas matplotlib > /dev/null

In [ ]:
import os
import math
from datetime import datetime, timedelta, time
import numpy as np
import cv2
from PIL import Image, ExifTags
import piexif
import pytz
import pandas as pd

from astropy.coordinates import EarthLocation, AltAz, get_sun
from astropy.time import Time
from astropy import units as u

# -------------- EXIF --------------
def extract_exif_datetime(image_path):
    try:
        img = Image.open(image_path)
        exif = img._getexif() or {}
        tagmap = {ExifTags.TAGS.get(t, t): v for t, v in exif.items()}
        for key in ("DateTimeOriginal", "DateTimeDigitized", "DateTime"):
            if key in tagmap:
                try:
                    return datetime.strptime(tagmap[key], "%Y:%m:%d %H:%M:%S")
                except:
                    pass
        return None
    except:
        return None

# -------------- Solar positions --------------
def compute_solar_positions_for_date(target_date, timezone="Asia/Bangkok"):
    tz = pytz.timezone(timezone)
    d = datetime.strptime(target_date, "%Y-%m-%d").date()
    start = tz.localize(datetime.combine(d, time(0,0)))
    return pd.date_range(start=start, end=start + timedelta(days=1)-timedelta(minutes=30), freq="30min", tz=tz)

def sun_positions_for_times(times, lat, lon):
    loc = EarthLocation(lat=lat*u.deg, lon=lon*u.deg)
    alts, azs = [], []
    for ts in times:
        ts_utc = ts.tz_convert('UTC')
        t_ast = Time(ts_utc.to_pydatetime())
        altaz = AltAz(obstime=t_ast, location=loc)
        sun_altaz = get_sun(t_ast).transform_to(altaz)
        alts.append(sun_altaz.alt.degree)
        azs.append(sun_altaz.az.degree)
    df = pd.DataFrame(index=times)
    df['elevation'] = alts
    df['azimuth'] = azs
    df['zenith'] = 90 - df['elevation']
    df['apparent_zenith'] = df['zenith']
    return df

# -------------- Normals --------------
def estimate_normals_from_image_gray(gray):
    gx = cv2.Sobel(gray.astype('float32'), cv2.CV_32F, 1, 0, 3)
    gy = cv2.Sobel(gray.astype('float32'), cv2.CV_32F, 0, 1, 3)
    gx /= (np.percentile(np.abs(gx),99)+1e-6)
    gy /= (np.percentile(np.abs(gy),99)+1e-6)
    nz = np.ones_like(gx)
    nx = -gx
    ny = -gy
    n = np.stack([nx, ny, nz], axis=2)
    n /= np.linalg.norm(n,axis=2,keepdims=True)+1e-8
    return n

# -------------- Color temp --------------
def color_temp_from_sun_elevation(e):
    e = max(min(e,90), -90)
    if e <= -6:
        return 4000
    if e < 0:
        t = (e+6)/6
        return 3000*(1-t) + 4500*t
    t = e/90
    return 4500*(1-t) + 6500*t

def apply_color_temperature(image_bgr, kelvin):
    def kelvin_to_rgb(k):
        t = k/100
        if t <= 66:
            r = 255
        else:
            r = 329.698727446*((t-60)**-0.1332047592)
        if t <= 66:
            g = 99.47*math.log(t)-161.12
        else:
            g = 288.122*((t-60)**-0.0755148492)
        if t >= 66:
            b = 255
        elif t <= 19:
            b = 0
        else:
            b = 138.5177*math.log(t-10)-305.0448
        return (r/255, g/255, b/255)
    r,g,b = kelvin_to_rgb(kelvin)
    out = image_bgr.astype('float32')
    out[:,:,0]*=b
    out[:,:,1]*=g
    out[:,:,2]*=r
    return np.clip(out,0,255).astype('uint8')

# -------------- Relighting --------------
def relight_image(image_bgr, normals, sun_az_deg, sun_el_deg,
                  ambient=0.25, diffuse_multiplier=1.3,
                  specular_power=80, specular_strength=0.03):
    el = math.radians(sun_el_deg)
    az = math.radians(sun_az_deg)
    sx = math.sin(az)*math.cos(el)
    sy = -math.sin(el)
    sz = math.cos(az)*math.cos(el)
    sun = np.array([sx,sy,sz],dtype='float32')
    sun /= np.linalg.norm(sun)+1e-8

    dot = np.maximum(0,(normals*sun.reshape(1,1,3)).sum(2))
    shading = ambient + diffuse_multiplier*dot
    shading = np.clip(shading,0,3)

    view = np.array([0,0,1],dtype='float32')
    hlf = view+sun
    hlf /= np.linalg.norm(hlf)+1e-8
    spec = np.maximum(0,(normals*hlf.reshape(1,1,3)).sum(2))
    spec = specular_strength*(spec**specular_power)

    out = image_bgr.astype('float32')/255
    out = out*shading[:,:,None]
    out = np.clip(out+spec[:,:,None],0,1)*255
    return out.astype('uint8')

# -------------- Sky + Sun overlay --------------
def overlay_sky_and_sun(image_bgr, rel_az, sun_el, intensity=0.6):
    h,w = image_bgr.shape[:2]
    cx, cy = w/2, h/3
    px = int(cx + (w/2)*math.tan(math.radians(rel_az))*0.5)
    py = int(cy - (h/2)*math.sin(math.radians(sun_el))*0.9)

    top = np.array([120,180,255],dtype='float32')
    bot = np.array([200,160,120],dtype='float32')
    t = np.clip((sun_el+10)/70,0,1)
    top *= (0.6+0.4*t)
    bot *= (0.7+0.3*t)
    sky = np.zeros_like(image_bgr,dtype='float32')
    for y in range(h):
        a = y/(h-1)
        sky[y] = top*(1-a) + bot*a

    rr = int(max(12, min(w,h)*(0.03 + (0.5-abs(sun_el)/180)*0.05)))
    Y,X=np.ogrid[:h,:w]
    dist = np.sqrt((X-px)**2+(Y-py)**2)
    sun_mask = np.exp(-(dist/(rr+1e-8))**2)
    sun_col = np.array([255,240,200],dtype='float32')*(0.2+2*max(0,math.sin(math.radians(sun_el))))
    sun_layer = np.zeros_like(sky)
    for c in range(3): sun_layer[:,:,c]=sun_col[c]*sun_mask

    out = image_bgr.astype('float32')
    sky_alpha = np.linspace(0.7,0.15,h).reshape(h,1)
    sky_alpha = np.repeat(sky_alpha,w,axis=1)[:,:,None]
    out = out*(1-sky_alpha*intensity) + sky*sky_alpha*intensity
    out = out + sun_layer*0.6
    return np.clip(out,0,255).astype('uint8')

# -------------- Main --------------
def process_day(image_path, camera_azimuth_deg, target_date, lat, lon, outdir):
    os.makedirs(outdir,exist_ok=True)
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    normals = estimate_normals_from_image_gray(gray)

    times = compute_solar_positions_for_date(target_date)
    sol = sun_positions_for_times(times, lat, lon)

    for ts,row in sol.iterrows():
        sun_az = row['azimuth']
        sun_el = row['elevation']
        rel_az = (sun_az - camera_azimuth_deg + 360) % 360
        if rel_az>180: rel_az -= 360

        shaded = relight_image(img, normals, rel_az, sun_el)
        comp = overlay_sky_and_sun(shaded, rel_az, sun_el)
        comp = apply_color_temperature(comp, color_temp_from_sun_elevation(sun_el))

        if sun_el < -6:
            comp = (comp.astype('float32')*0.15).astype('uint8')

        fname = ts.strftime('%Y%m%d_%H%M')
        cv2.imwrite(os.path.join(outdir,f"relit_{fname}.jpg"), comp)

    print("Done.")

print("Colab relighting pipeline loaded.")

In [ ]:
process_day(
    "/content/IMG_20251114_074800.jpg",
    90.47,
    "2025-11-18",
    20.99388016530414,
    105.86847156589005,
    "/content/",
)
